# Задание 1


Выберите 5 языков в википедии (не тех, что использовались в семинаре). Скачайте по 10 случайных статей для каждого языка. Предобработайте тексты, удаляя лишние теги/отступы/разделители (если они есть). Разделите тексты на предложения и создайте датасет, в котором каждому предложению соответствует язык. Кластеризуйте тексты, используя эбмединг модель из прошлого семинара и любой алгоритм кластеризации. Проверьте качество кластеризации с помощь метрики ARI. Отдельно проанализируйте 3 ошибочно кластеризованных текста (если такие есть).

In [1]:
import wikipedia
import numpy as np
import nltk
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score
from tqdm.notebook import tqdm
import re
from collections import Counter
import pandas as pd


nltk.download('punkt')
nltk.download('punkt_tab')

2025-10-08 10:32:40.414589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759908760.430316   36566 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759908760.435001   36566 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759908760.447533   36566 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759908760.447556   36566 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759908760.447558   36566 computation_placer.cc:177] computation placer alr

True

## 1. Загружаем данные

In [2]:
def load_with_disambiguation(page):
    # Функция для загрузки страницы с обработкой неоднозначностей
    try:
        p = wikipedia.page(page, auto_suggest=False)
    except wikipedia.exceptions.DisambiguationError as e:
        random_option = np.random.choice(e.options)
        p = load_with_disambiguation(random_option)
    return p


def get_texts_for_lang(lang, n=10):
    # Функция для скачивания n случайных статей для заданного языка
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)

    for page_name in tqdm(pages, desc=f'Loading pages for {lang}'):
        try:
            page = load_with_disambiguation(page_name)
            # Убираем заголовки секций и лишние переносы строк
            content = re.sub(r'==.*?==', '', page.content)
            content = re.sub(r'\n+', '\n', content).strip()
            if content:  # Добавляем только если есть контент
                wiki_content.append(content)
        except Exception as e:
            print(
                f'Skipping page {page_name} in language {lang} due to error: {e}')
            continue
    return wiki_content

In [3]:
%%time
# Французский, немецкий, испанский, шведский, датский
langs = ['fr', 'de', 'es', 'sv', 'da']
wiki_texts = {}

for lang in langs:
    wiki_texts[lang] = get_texts_for_lang(lang, n=10)
    print(f'Downloaded {len(wiki_texts[lang])} articles for language: {lang}')

Loading pages for fr:   0%|          | 0/10 [00:00<?, ?it/s]

/home/futyn/miniconda3/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/futyn/miniconda3/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Downloaded 10 articles for language: fr


Loading pages for de:   0%|          | 0/10 [00:00<?, ?it/s]

Downloaded 10 articles for language: de


...

Downloaded 10 articles for language: es


Loading pages for sv:   0%|          | 0/10 [00:00<?, ?it/s]

Downloaded 10 articles for language: sv


Loading pages for da:   0%|          | 0/10 [00:00<?, ?it/s]

Skipping page Nørreby in language da due to error: Page id "Nørreby (Femø Sogn)" does not match any pages. Try another id!
Downloaded 9 articles for language: da
CPU times: user 1.31 s, sys: 146 ms, total: 1.46 s
Wall time: 4min 27s


## 2. Готовим датасет

In [4]:
lang_map_nltk = {
    'fr': 'french',
    'de': 'german',
    'es': 'spanish',
    'sv': 'swedish',
    'da': 'danish'
}

sentences = []
labels = []

for lang, texts in wiki_texts.items():
    for text in texts:
        sents = nltk.sent_tokenize(text, language=lang_map_nltk[lang])
        sentences.extend(sents)
        labels.extend([lang] * len(sents))

print(f'Total sentences: {len(sentences)}')

Total sentences: 1122


In [6]:
sentences[:5]

['The Ice Warriors (Les Guerriers de Glace) est le trente-neuvième épisode de la première série de la série télévisée britannique de science-fiction Doctor Who, diffusé pour la première fois en six parties hebdomadaires du 11 novembre au 16 décembre 1967, il montre pour la première fois les "Guerriers des glaces" des ennemis récurrents du Docteur.',
 'Sur les six parties de cet épisode, deux sont manquantes.',
 "Le TARDIS arrive sur la Terre du futur en proie à une nouvelle ère glaciaire, et les voyageurs trouvent le chemin vers une base scientifique où une créature à l'apparence d'un humain géant a été trouvé prisonnier des glaces.",
 'Ces étranges créatures se trouvent être des guerriers venus de Mars et prêts à tout pour retrouver leur planète.',
 "Patrick Troughton — Le Docteur\nFrazer Hines — Jamie McCrimmon\nDeborah Watling  — Victoria Waterfield\nPeter Barkworth — Clent\nPeter Sallis — Penley\nWendy Gifford — Miss Garrett\nAngus Lennie — Storr\nPeter Diamond — Davis\nGeorge Wari

## 3. Эмбеддинги

Будем использовать уже полюбившуюся инструктивную E5-модель.

In [7]:
model_name = 'intfloat/multilingual-e5-large-instruct'
model = SentenceTransformer(model_name)

In [8]:
task_instruction = 'Identify the language of the text'

formatted_sentences = [
    f'Instruct: {task_instruction}\nQuery: {s}' for s in sentences]

In [9]:
embeddings = model.encode(
    formatted_sentences,
    normalize_embeddings=True,
    show_progress_bar=True)

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

## 4. Кластеризация

В прошлый раз лучше всего себя показал AgglomerativeClustering. Учитывая, что мы можем задать количество кластеров (по количеству языков), это представляется лучшим выбором.

In [10]:
%%time
clusterizer = AgglomerativeClustering(n_clusters=len(langs), linkage='ward')
cluster_labels = clusterizer.fit_predict(embeddings)

CPU times: user 260 ms, sys: 8.41 ms, total: 269 ms
Wall time: 275 ms


In [11]:
ari_score = adjusted_rand_score(labels, cluster_labels)
print(f'Adjusted Rand Index (ARI): {ari_score:.4f}')

Adjusted Rand Index (ARI): 0.7086


## 5. Анализ ошибок

In [12]:
# Датафрейм, чтобы было удобнее анализировать
df = pd.DataFrame({'sentence': sentences,
                   'true_label': labels,
                   'cluster_id': cluster_labels})

# Находим наиболее частый язык для каждого кластера
cluster_to_lang_map = {}
for i in range(len(langs)):
    # Фильтруем по кластеру и находим самый частый язык
    most_common_lang = df[df['cluster_id'] == i]['true_label'].mode()[0]
    cluster_to_lang_map[i] = most_common_lang

# Применяем маппинг для получения предсказанных языков
df['predicted_label'] = df['cluster_id'].map(cluster_to_lang_map)

In [13]:
# Считаем количество верных и неверных предсказаний
correct_predictions = (df['true_label'] == df['predicted_label']).sum()
incorrect_predictions = len(df) - correct_predictions

print(f"Total sentences: {len(df)}")
print(f"Correctly clustered: {correct_predictions}")
print(f"Incorrectly clustered: {incorrect_predictions}")

Total sentences: 1122
Correctly clustered: 1043
Incorrectly clustered: 79


In [14]:
# Находим все ошибочно кластеризованные тексты
misclassified = df[df['true_label'] != df['predicted_label']]

print("Analysis of erroneously clustered texts:\n")

for lang in langs:
    lang_errors = misclassified[misclassified['true_label'] == lang]
    if not lang_errors.empty:
        print(f"--- Language: {lang.upper()} ---")
        print(f"Found {len(lang_errors)} errors. Examples:")
        for _, row in lang_errors.head(3).iterrows():
            print(f"  - Sentence: '{row['sentence']}'")
            print(
                f"    - True: {row['true_label']}, Predicted as: {row['predicted_label']}\n")
    else:
        print(f"--- Language: {lang.upper()} ---")
        print("No clustering errors found for this language.\n")

Analysis of erroneously clustered texts:

--- Language: FR ---
Found 5 errors. Examples:
  - Sentence: '=
2003 : Evil Never Dies
2005 : Hell on Earth
2008 : An Overdose of Death...
2011 : Conjure and Command
2013 : Chemistry of Conciousness
2019  : Primal Future: 2019
=
1999 : Radiation Sickness
2002 : Critical Mass
2004 : Promo 2004
=
2001 : Toxic Holocaust - Orichniki
2002 : Implements of Mass Destruction Nuclear Apocalypse:666
2004 : Outbreak of Evil
2004 : Thrashbeast from Hell
2005 : Blasphemy, Mayhem, War
2005 : HRPS Vol.1'
2006 : Don't Burn the Witch
2008 : Speed n' Spikes Vol.'
    - True: fr, Predicted as: da

  - Sentence: 'J.-C.'
    - True: fr, Predicted as: da

  - Sentence: '].'
    - True: fr, Predicted as: da

--- Language: DE ---
Found 34 errors. Examples:
  - Sentence: '„I have spoken“.'
    - True: de, Predicted as: da

  - Sentence: 'XLI.'
    - True: de, Predicted as: da

  - Sentence: 'The Edwin Mellen Press, Lewiston/Queenston/Lampeter 1993.'
    - True: de, Pred

В целом, эмбеддинги достаточно качественные и кластеризуются очень неплохо. Ошибки в большинстве случаев связаны с шумом и мусором, не очень понятно, как его вычистить: названия произведений, ссылки на дополнительную литературу на английском и так далее. В одном из немецких текстов, видимо, попались какие-то цитаты на английском.

По-настоящему путаются только похожие языки: датский и шведский. Мы видим много валидных шведских предложений, которые классифицированы как датские. Это ожидаемо, так как языки родственные и сильно похожи и, видимо, фичи их эмбеддингов во многом совпадают. Забавно, что обратной ситуации не встречается, да и вообще кластер датского языка перетягивает на себя все ошибки, хотя все сами датские предложения кластеризовались верно. По всей видимости, сюда попадают все предложения, для которых не удалось выявить уверенных паттернов для отнесения к другим кластерам.

# Задание 2

Загрузите корпус `annot.opcorpora.no_ambig_strict.xml.bz2` с OpenCorpora. Найдите в корпусе самые частотные морфологически омонимичные словоформы (те, которым соответствует разный грамматический разбор в разных предложениях). Также найдите словоформы с самых большим количеством вариантов грамматических разборов.

## 1. Загружаем и открываем корпус

In [15]:
!wget http://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2025-10-08 11:27:12--  http://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2
Resolving opencorpora.org (opencorpora.org)... 172.67.163.210, 104.21.15.199, 2606:4700:3031::ac43:a3d2, ...
Connecting to opencorpora.org (opencorpora.org)|172.67.163.210|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2 [following]
--2025-10-08 11:27:12--  https://opencorpora.org/files/export/annot/annot.opcorpora.no_ambig_strict.xml.bz2
Connecting to opencorpora.org (opencorpora.org)|172.67.163.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1435452 (1.4M) [application/x-bzip2]
Saving to: ‘annot.opcorpora.no_ambig_strict.xml.bz2’

annot.opcorpora.no_ 100%[===================>]   1.37M  3.95MB/s    in 0.3s    

2025-10-08 11:27:13 (3.95 MB/s) - ‘annot.opcorpora.no_ambig_strict.xml.bz2’ saved [1435452/1435452]



In [16]:
import bz2

with bz2.open('annot.opcorpora.no_ambig_strict.xml.bz2', 'rb') as f_in, open('annot.opcorpora.no_ambig_strict.xml', 'wb') as f_out:
    f_out.write(f_in.read())

In [17]:
from lxml import etree
from collections import defaultdict
from tqdm.notebook import tqdm


open_corpora = etree.fromstring(
    open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

## 2. Собираем и структурируем данные

In [18]:
analyses_by_word = defaultdict(list)

# Итерируемся по всем предложениям в корпусе
for sentence in tqdm(open_corpora.xpath('//tokens')):
    for token in sentence.xpath('token'):
        # Получаем словоформу
        word_form = token.xpath('@text')
        # Получаем грамматическую информацию
        gram_info = token.xpath('tfr/v/l/g/@v')

        # Проверяем, что оба значения существуют
        if word_form and gram_info:
            # Сохраняем информацию в виде кортежа, чтобы ее можно было добавить
            # в set
            analyses_by_word[word_form[0].lower()].append(
                tuple(sorted(gram_info)))

  0%|          | 0/10971 [00:00<?, ?it/s]

## 3. Самые частотные омонимичные формы

In [21]:
homonym_frequencies = {}

for word, analyses in analyses_by_word.items():
    if len(set(analyses)) > 1:
        homonym_frequencies[word] = len(analyses)

# Сортируем слова по убыванию их частоты
sorted_homonyms_by_freq = sorted(
    homonym_frequencies.items(),
    key=lambda item: item[1],
    reverse=True)

print("Top 20 most frequent morphologically homonymous word forms:")
for word, freq in sorted_homonyms_by_freq[:20]:
    print(f"- '{word}': {freq} occurrences")
    for analysis in set(analyses_by_word[word]):
        print(f"    - Analysis: {', '.join(analysis)}")

Top 20 most frequent morphologically homonymous word forms:
- 'в': 2059 occurrences
    - Analysis: Abbr, Fixd, NOUN, gent, inan, masc, sing
    - Analysis: PREP
- 'на': 786 occurrences
    - Analysis: PRCL
    - Analysis: PREP
- 'с': 613 occurrences
    - Analysis: PRCL
    - Analysis: PREP
- 'и': 574 occurrences
    - Analysis: CONJ
    - Analysis: PRCL
- 'о': 213 occurrences
    - Analysis: INTJ
    - Analysis: PREP
- 'году': 115 occurrences
    - Analysis: NOUN, inan, loc2, masc, sing
    - Analysis: NOUN, datv, inan, masc, sing
- 'а': 113 occurrences
    - Analysis: CONJ
    - Analysis: INTJ
- 'этом': 104 occurrences
    - Analysis: ADJF, Anph, Apro, Subx, loct, masc, sing
    - Analysis: ADJF, Anph, Apro, Subx, loct, neut, sing
    - Analysis: NPRO, loct, neut, sing
- 'россии': 91 occurrences
    - Analysis: Geox, NOUN, Sgtm, datv, femn, inan, sing
    - Analysis: Geox, NOUN, Sgtm, femn, gent, inan, sing
    - Analysis: Geox, NOUN, Sgtm, femn, inan, loct, sing
- 'было': 89 occurr

В основном это служебные слова, которые могут относиться к разным частям речи: союзы, предлоги, частицы и местоимения. Есть только два частотных падежных омонима: **году** и **России**.

## 4. Самые грамматически неоднозначные слова

In [22]:
homonym_variety = {}

for word, analyses in analyses_by_word.items():
    num_unique_analyses = len(set(analyses))
    # Нас интересуют только слова с >1 вариантом разбора
    if num_unique_analyses > 1:
        homonym_variety[word] = num_unique_analyses

# Сортируем слова по убыванию количества уникальных разборов
sorted_homonyms_by_variety = sorted(
    homonym_variety.items(),
    key=lambda item: item[1],
    reverse=True)

print("\nTop 20 word forms with the most grammatical analyses:")
for word, num_analyses in sorted_homonyms_by_variety[:20]:
    print(f"- '{word}': {num_analyses} unique analyses")
    for analysis in set(analyses_by_word[word]):
        print(f"    - Analysis: {', '.join(analysis)}")


Top 20 word forms with the most grammatical analyses:
- 'сша': 6 unique analyses
    - Analysis: Abbr, Fixd, GNdr, Geox, NOUN, Pltm, datv, inan, plur
    - Analysis: Abbr, Fixd, GNdr, Geox, NOUN, Pltm, gent, inan, plur
    - Analysis: Abbr, Fixd, GNdr, Geox, NOUN, Pltm, accs, inan, plur
    - Analysis: Abbr, Fixd, GNdr, Geox, NOUN, Pltm, inan, loct, plur
    - Analysis: Abbr, Fixd, GNdr, Geox, NOUN, Pltm, ablt, inan, plur
    - Analysis: Abbr, Fixd, GNdr, Geox, NOUN, Pltm, inan, nomn, plur
- 'кино': 5 unique analyses
    - Analysis: Fixd, NOUN, gent, inan, neut, sing
    - Analysis: Fixd, NOUN, accs, inan, neut, sing
    - Analysis: Fixd, NOUN, datv, inan, neut, sing
    - Analysis: Fixd, NOUN, inan, neut, nomn, sing
    - Analysis: Fixd, NOUN, inan, loct, neut, sing
- 'евро': 5 unique analyses
    - Analysis: Fixd, NOUN, gent, inan, neut, sing
    - Analysis: Fixd, NOUN, datv, inan, neut, sing
    - Analysis: Fixd, NOUN, accs, inan, neut, sing
    - Analysis: Fixd, NOUN, inan, neut, 

Здесь встречаем омонимы трёх типов. Во-первых, неизменяемые существительные (в т.ч. сокращения): *США*, *кино*, *НАТО*, *евро*. Во-вторых - падежные омографы и омонимы, где много форм с совпадающими окончаниями: *путь*, *лицо*, *компания*, *версия*. Ну и, наконец, служебные слова, которые допускают разный анализ (частично пересекается с первым списком): *какой*, *что*, *это*.

# Задание 3
Загрузите один и з файлов корпуса Syntagrus - https://github.com/UniversalDependencies/UD_Russian-SynTagRus/tree/master (можно взять тестовый)

Преобразуйте все разборы предложений в графовые структуры через DependencyGraph, выберите 3 любых отношения и для каждого найдите топ-5 самых встречаемых пар слов, связанных этим отношением. 

Для самой частотной пары слов в каждом из отношений вытащите все подзависимые слова для каждого из них во всех предложениях (используя `flatten(get_subtree(d.nodes, index_of_a_word)` и сортируя результат по порядку слов в предложениях, аналогично тому как я делал с summaries только у вас будет два слова) 
В итоге у вас должен получится что-то такое:

```
### отношение
relation_name

### топ 5 пар слов связанных этим отношением
(word1, word2), (word3, word4), (word5, word6), (word7, word8), (word9, word10)

### подзависимые для самого частотного
(subword word1 subword, word2 subword subword)

... (и так три раза)
```


In [1]:
import os
from nltk.parse import DependencyGraph
from collections import Counter, defaultdict
from tqdm.notebook import tqdm

In [2]:
# Рекурсивная функция для получения всех узлов поддерева
def get_subtree(nodes, node):
    # Исключаем пунктуацию
    if not nodes[node]['deps'] or all(
            rel == 'punct' for rel in nodes[node]['deps']):
        return [node]
    else:
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps']
                         if rel != 'punct'
                         for dep in nodes[node]['deps'][rel]]


# Рекурсивная функция для преобразования вложенных списков
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

## 1. Загружаем и парсим данные

In [3]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu -O ru_syntagrus-ud-test.conllu

--2025-10-08 13:55:39--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15253211 (15M) [text/plain]
Saving to: ‘ru_syntagrus-ud-test.conllu’

ru_syntagrus-ud-tes 100%[===================>]  14.55M  12.9MB/s    in 1.1s    

2025-10-08 13:55:42 (12.9 MB/s) - ‘ru_syntagrus-ud-test.conllu’ saved [15253211/15253211]



In [4]:
graphs = []
with open('ru_syntagrus-ud-test.conllu', 'r', encoding='utf-8') as f:
    sents = f.read().split('\n\n')

for sent in tqdm(sents):
    # Отфильтровываем строки с метаинформацией и пустые строки
    tree_lines = [line for line in sent.split('\n') if line and line[0] != '#']
    if not tree_lines:
        continue

    try:
        graph = DependencyGraph(
            '\n'.join(tree_lines),
            top_relation_label='root')
        graphs.append(graph)
    except Exception as e:
        print(f"Skipping sentence due to parsing error: {e}")
        pass

print(f"Successfully parsed {len(graphs)} sentences.")

  0%|          | 0/8801 [00:00<?, ?it/s]

Skipping sentence due to parsing error: 
Skipping sentence due to parsing error: 
Skipping sentence due to parsing error: 
Skipping sentence due to parsing error: 
Successfully parsed 8796 sentences.


## 2. Собираем статистику по словам для отношений

Мы будем рассматривать самое стандартное: субъект, объект и модификатор.

In [5]:
relations_data = defaultdict(Counter)
relations_to_find = ['nsubj', 'obj', 'amod']

for graph in tqdm(graphs):
    for node_index, node in graph.nodes.items():
        # Пропускаем корневой узел
        if node['head'] is None:
            continue

        relation = node['rel']
        if relation in relations_to_find:
            head_node = graph.nodes[node['head']]

            # Убеждаемся, что у обоих слов есть лемма
            if head_node.get('lemma') and node.get('lemma'):
                head_lemma = head_node['lemma']
                dep_lemma = node['lemma']
                relations_data[relation][(head_lemma, dep_lemma)] += 1

  0%|          | 0/8796 [00:00<?, ?it/s]

## 3. Извлекаем поддеревья

In [6]:
for relation in relations_to_find:
    print(f"### отношение\n{relation}\n")

    top_5_pairs = relations_data[relation].most_common(5)
    print("### топ 5 пар слов связанных этим отношением")
    print(', '.join([f"({p[0][0]}, {p[0][1]})" for p in top_5_pairs]))

    # Находим подзависимые для самой частой пары
    print("\n### подзависимые для самого частотного")

    if not top_5_pairs:
        print("(нет данных для анализа)\n")
        print("-" * 40)
        continue

    top_head_lemma, top_dep_lemma = top_5_pairs[0][0]

    # Итерируемся по корпусу снова, чтобы найти примеры
    found_examples = 0
    for graph in graphs:
        for node_index, node in graph.nodes.items():
            if node['rel'] == relation and node['head'] is not None:
                head_node = graph.nodes[node['head']]

                # Ищем точное совпадение по леммам
                if head_node.get('lemma') == top_head_lemma and node.get(
                        'lemma') == top_dep_lemma:

                    # Получаем поддеревья для обоих слов
                    head_subtree_indices = sorted(
                        flatten(
                            get_subtree(
                                graph.nodes,
                                head_node['address'])))
                    dep_subtree_indices = sorted(
                        flatten(get_subtree(graph.nodes, node['address'])))

                    # Преобразуем индексы обратно в слова
                    head_subtree_words = ' '.join(
                        [graph.nodes[i]['word'] for i in head_subtree_indices if graph.nodes[i]['word'] is not None])
                    dep_subtree_words = ' '.join(
                        [graph.nodes[i]['word'] for i in dep_subtree_indices if graph.nodes[i]['word'] is not None])

                    print(f"({head_subtree_words}, {dep_subtree_words})")
                    found_examples += 1

    if found_examples == 0:
        print("(не найдено предложений с данной парой)")

    print("\n" + "-" * 40 + "\n")

### отношение
nsubj

### топ 5 пар слов связанных этим отношением
(сказать, я), (мочь, я), (видеть, я), (идти, речь), (касаться, что)

### подзависимые для самого частотного
(Я сказал эти сметы пока отложите да да, Я)
(И я бы сказал что Евангелие укрепляет нашу человечность, я)
(Я просто скажу что по-моему украинское общество сейчас на распутье, Я)
(сказал я, я)
(сказал я, я)
(Я бы сказал что определенным триггером стала отставка Черномырдина с поста премьер-министра, Я)
(Я это сказала вслух и еще несколько человек признались что то же самое, Я)
(Я сказала что живу на Садовой улице совсем недалеко от их бывшего дома, Я)
(сказал я поглядывая на Смерчева снизу и сбоку, я)
(сказал я, я)
(сказал я жене, я)
(сказал я жене, я)
(сказал я пытаясь сдвинуть его со своего пути, я)
(сказал я себе самому, я)
(Уклоняясь от прямого ответа я сказал что я латыш, я)
(сказал я, я)
(сказал я, я)
(сказал я к ней повернувшись, я)
(сказал я не выдержав и перешел на другую сторону, я)
(сказал я себе самому, я

В целом результат достаточно правдоподобный, если помнить, что основу для корпуса составляют новостные тексты, в которых часто упоминаются даты, а также используются соответствующие штампы.